# A simple trace

The `trace` module offers the function `trace` for small models. It provides easy to use functionality with defaults.

## 1. imports

In [1]:
import itertools
from typing import Optional

import simpy
from sim_tools.trace import trace
from sim_tools.distributions import Exponential

## 2. A trace using default colouring

In this first example we simulate patient arrivals to a treatment facility.  Arrivals are at random with a mean inter-arrival time of 5 minutes.
Rather than using python's print command we instead make use of `trace`. We pass the following arguments:

1. `time`: the current simulation time
2. `debug`: if we toggle to `False` this hides the trace or `True` to show (default = False)
3. `msg`: the string message to display. This can include emoji 
4. `process_id`: an optional string to identify the process. Ideally this should be unique to aid debugging.

In [2]:
def patient_generator(env: simpy.Environment, dist: Exponential, debug: Optional[bool]=False):
    """Generate patient arrivals to the treatment clinic"""
    for patient_count in itertools.count(1):
        # sample inter-arrival time
        iat = dist.sample()
        yield env.timeout(iat)
        trace(time=env.now, debug=debug, msg="new arrival 🤒", identifier=patient_count)


In [3]:
# script to run model
DEBUG = True
SEED = 42
arrival_dist = Exponential(5.0, random_seed=SEED)
env = simpy.Environment()
env.process(patient_generator(env, arrival_dist, DEBUG))
env.run(50.0)

[12.02]:<event 1>: new arrival 🤒

[23.70]:<event 2>: new arrival 🤒

[35.63]:<event 3>: new arrival 🤒

[37.02]:<event 4>: new arrival 🤒

[37.46]:<event 5>: new arrival 🤒

[44.72]:<event 6>: new arrival 🤒

## 3. Configure colouring of output


In `trace` the `config` parameter is a user settable dictionary object. It can be used to change the colour of text in the trace.  We will first show a simple demonstration of setting options. and then use to illustrate how it is useful in practice.

The default`config` is

```python
config = {
    "class":None, 
    "class_colour":"bold blue", 
    "time_colour":'bold blue', 
    "time_dp":2,
    "message_colour":'black',
    "tracked":None
}
```

* `class`: a string representing the class or type of trace event occuring.  This could be a process type for example, "patient", "stroke patient" or "arrival" or "treatment".
* `class_colour`: choose a colour to display the class name e.g. "green" or "bold green"
* `time_colour`: choose a colour to display the time
* `time_dp`: choose the number of decimal places for time (default=2)
* `message_color`: colour of the message text
* `tracked`: a list containing identifiers (e.g. `[1, 2, 25]`) that limits what is tracked.  Works with `identiifier` parameters of `trace`

> Note: you do not need to set all of the parameters.  Just set what you need and the defaults will be used for other parameters.

In [4]:
def get_config():
    """Returns a custom trace configuration"""
    config = {
        "class":"Patient", 
        "class_colour":"green", 
        "time_colour":'bold black',
        "message_colour":"red"
    }
    return config


def patient_generator(env: simpy.Environment, dist: Exponential, debug: Optional[bool]=False):
    """Generate patient arrivals to the treatment clinic"""
    for patient_count in itertools.count(1):
        # sample inter-arrival time
        iat = dist.sample()
        yield env.timeout(iat)
        trace(time=env.now, 
              debug=debug, 
              msg="new arrival 🤒", 
              identifier=patient_count,
              config=get_config())

In [5]:
# script to run model
DEBUG = True
SEED = 42
arrival_dist = Exponential(5.0, random_seed=SEED)
env = simpy.Environment()
env.process(patient_generator(env, arrival_dist, DEBUG))
env.run(50.0)

[12.02]:<Patient 1>: new arrival 🤒

[23.70]:<Patient 2>: new arrival 🤒

[35.63]:<Patient 3>: new arrival 🤒

[37.02]:<Patient 4>: new arrival 🤒

[37.46]:<Patient 5>: new arrival 🤒

[44.72]:<Patient 6>: new arrival 🤒